# Using Exploratory Data Analysis to Generate Movie Insights for Microsoft

## 1. Business Understanding

### a) Problem Statement

> Microsoft company, having seen how well the movie industry is doing, have spotted a business opportunity in the same. The company is therefore looking to create a movie studio. However, this may pose a challenge because it is a very new development and the company does not have enough insight on how to go about it. 

> Since Microsoft have no experience in creating movies, this exploratory data analysis seeks to find what types of films are currently doing the best at the box office. The findings from this insight will be used to come up with recommendations on the right course of action for Microsoft to take following the best performing films.

### b) Defining the Metric for Success
This explatory analysis will be considered a success if the following results are achieved:
i) If the recommendations drawn from the conclusions are actionable and if they provide a well laid out step by step guide
ii)

## 2. Data Understanding

In [5]:
#Importing the relevant packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
#Loading datasets from our various sources i.e csv
bom_movies = pd.read_csv('bom.movie_gross.csv')
tmdb_movies = pd.read_csv('tmdb.movies.csv')
tn_movie_budgets = pd.read_csv('tn.movie_budgets.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'bom.movie_gross.csv'

In [ ]:
# Importing Sqlite3 Module
import sqlite3
# Making a connection between sqlite3
# database and Python Program
conn = sqlite3.connect('im.db')
 # Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
 # Creating cursor object using connection object
cursor = conn.cursor()
# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")
# printing all tables list
print(cursor.fetchall())



### a) Checking and Understanding the Data

In [ ]:
# For 'im.db' data, the relevant data to this analysis is from movie_basics and m0vie_ratings tables. Therefore, we check the two table
q = '''SELECT *
    FROM movie_basics;
    '''
pd.read_sql(q, conn)

In [ ]:
movie_basics = pd.read_sql(q, conn)
movie_basics

In [ ]:
#checking the movie_ratings dataset
q2 = '''SELECT *
     FROM movie_ratings;
     '''
pd.read_sql(q2, conn)

In [ ]:
movie_ratings = pd.read_sql(q2, conn)
movie_ratings

In [ ]:
# Determining the no. of records in each of our csv datasets
bom_movies

In [ ]:
# seeking further information from the bom_movies dataset (helps to gain better understanding components of the data)
bom_movies.info()

In [ ]:
#checking the tmdb_movies dataset
tmdb_movies

In [ ]:
#Dropping the column Unnamed because it is irrelevant to the dataset
tmdb_movies = tmdb_movies.drop(['Unnamed: 0'], axis = 1)
tmdb_movies

In [ ]:
#seeking further information from the tmdb_movies dataset
tmdb_movies.info()

In [ ]:
#checking the tn_movie_budgets dataset
tn_movie_budgets

In [ ]:
#seeking further information from the tn_movie_budgets dataset
tn_movie_budgets.info()

In [ ]:
# Checking whether each column has an appropriate datatype
bom_movies_df = (bom_movies)
bom_movies_df.dtypes

In [ ]:
tmdb_movies_df = (tmdb_movies)
tmdb_movies_df.dtypes

In [ ]:
tn_movie_budgets_df = (tn_movie_budgets)
tn_movie_budgets_df.dtypes

In [ ]:
'''Following the information observed from the datasets selected, some columns of several datasets seem to have missing values and inconsistent data'''

### b) Checking for Inconsistencies, duplicates and missing values

Having studied the data and realized several of them have missing values,so for this next step we check for missing values and other inconsistencies and possible duplicates we identify and handle them.For the relevant datasets, dropping the rows with the missing values is the most suitable solution. 

####  Duplicates

In [ ]:
# Checking for duplicates in movie_basics
movie_basics.duplicated().sum()

There are no duplicates in movie_basics. We therefore move on to the next dataset

In [ ]:
# Checking for duplicates in movie_ratings
movie_ratings.duplicated().sum()

Similarly, there are no duplicates in movie_ratings

In [ ]:
#Checking for duplicates in bom_movies
bom_movies.duplicated().value_counts()

In [ ]:
#Checking for duplicates in tmdb_movies
tmdb_movies.duplicated().value_counts()

In [ ]:
#Cheking for duplicates in tn_movie_budgets
tn_movie_budgets.duplicated().value_counts()

the tmdb_movies dataset has 1020 duplicates. In the next process, we will drop these columns

#### Missing Values

runtime_minutes column in movie_basics has null values in 31739 out of 146144 rows. In the next process, we will drop these rows since the remaining ones will still be sufficient for the analysis.

Genres column in movie_basics has 5408 rows with null values

In [ ]:
movie_ratings.isna().sum()

both columns in movie_ratings have no missing values

In [ ]:
#checking for missing values in bom_movies
bom_movies.isna().sum()

studio, domestic_gross, and foreign_gross columns contain rows with missing values. In the next step, the rows will be dropped since they are of insignificant effect to the whole analysis process.

In [ ]:
#checking for missing values in tmdb_movies
tmdb_movies.isna().sum()

tmdb_movies has no null values

In [ ]:
#checking for missing values in th_movie_budgets
tn_movie_budgets.isna().sum()

In [ ]:
tn_movie_budgets

tn_movie_budgets has no null values

## 3. Data Cleaning

The most prevalent issue with the dataset we have is missing values and duplicates. Therefore, in this process we are going to handle that by dropping the rows with the null values.

In [ ]:
# Dealing with duplicates in tmdb_movies
# Use keep=False to keep all duplicates and sort_values to put duplicates next to each other
tmdb = tmdb_movies
tmdb[tmdb.duplicated(keep=False)].sort_values(by='genre_ids')

In [ ]:
 # Remove duplicates
tmdb = tmdb.drop_duplicates()
tmdb.shape 
# Previously this was (26517, 9), now we have dropped duplicate rows

In [ ]:
# Recheck for duplicates
tmdb.duplicated().value_counts()

In [ ]:
tmdb

Duplicates no longer exist in tmdb dataset

In [ ]:
movie_basics = movie_basics.dropna()
movie_basics

In [ ]:
movie_basics.isna().sum()

In [ ]:
# Dropping missing values in bom_movies
bom_movies = bom_movies.dropna()
bom_movies

In [ ]:
# Rechecking for missing values 
bom_movies.isna().sum()

## 4. Merging Datasets

 Having carried out the necessary data checks and cleaning processes, the data is ready to be analyzed. This process involves merging the different sets oof data we have into a single dataset.

In [ ]:
# First join the two SQl tables, movie_basics and movie_ratings
cleaned_data1 = '''SELECT movie_basics.runtime_minutes,movie_basics.genres,
                   movie_ratings.averagerating, movie_ratings.numvotes
                   FROM (movie_basics ,movie_ratings) AS cleaned_data1
                   WHERE  movie_basics.movie_id =movie_ratings.movie_id;
                '''
pd.read_sql(cleaned_data1, conn)

In [ ]:
cleaned_data1 = pd.read_sql(cleaned_data1, conn)
cleaned_data1

In [ ]:
cleaned_data1.isna().sum()

We now have a single dataset with the relevant columns from the two datasets. However, since the number of rows and columns were unmatched, we have ended up with several rows in runtime_minutes column. We will need to drop the rows with missing values again.

In [ ]:
cleaned_data1.dropna()

In [ ]:
# Now we merge the three csv datasets; bom_movies, tmdb and tn_movie_budgets
df = pd.concat([bom_movies, tmdb, tn_movie_budgets])
df

With respect to what the analysis seeks to achieve, we only need a few columns from the merged dataset, that is the domestic_gross, foreign_gross, popularity, vote_average, and production_budget. The other columns proving irrelevant to this analysis, will therefore be dropped.

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['studio','year','Unnamed: 0','genre_ids', 'id', 'original_language', 'original_title', 'release_date', 'vote_count', 'movie', 'worldwide_gross'], axis=1, inplace=True)

In [ ]:
df

There are several columns with null values, we drop popularity and vote_average colums and replace the null values in production_budget because the data is very significant to the analysis.

In [ ]:
df.drop(['popularity', 'vote_average'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Replacing the null values in production_budget
df.production_budget.fillna(0,inplace=True)

In [ ]:
df

In [ ]:
# Replace the null values in foreign_gross with zero
df.foreign_gross.fillna(0,inplace=True)

In [ ]:
df

In [ ]:
# Drop rows with null values
df = df.dropna()

From our dataset, it seems the budget_column was from a dataframe with not titles, so dropping the rows with null values means we end up with no data for production_budget. For this reason, the column has been rendered irrelevant to our analysis and will therefore be dropped.

In [ ]:
df.drop(['production_budget'], axis=1, inplace=True)

In [ ]:
df

Our final dataframe (df), has 2007 rows and 3 columns. We now have two datasets that we are going to explore and analyze, cleaned_data1 and df

## 5. Explatory Data Analysis

> The easy solution is nice because it is, well, easy, but you should never allow those results to hold the day. You should always be thinking of ways to challenge the results, especially if those results comport with your prior expectation.






In [ ]:
# Reviewing the Solution 
#

## 9. Follow up questions

> At this point, we can refine our question or collect new data, all in an iterative process to get at the truth.



### a). Did we have the right data?

### b). Do we need other data to answer our question?

### c). Did we have the right question?